# From Experiments to Data
## Computational Methods in Psychology (and Neuroscience)
### Psychology 4215/7215 --- Fall 2021
By: Per B. Sederberg, PhD



# Lesson Objectives

Upon completion of this lesson, students should have learned:

1. How to polish up a complete experiment
2. How SMILE stores data
3. How to read in slog files
4. An introduction to Pandas


# Review list generation solution

- Let's go through the list gen code you'll use for your next assignment!

# Subject Info

- When collecting data from a lot of participants in a lab setting, you need an easy way to enter subject information once the experiment has started.

In [6]:
from smile.common import *
from smile.startup import InputSubject

exp = Experiment(show_splash=False, resolution=(1024,768))

InputSubject('Flanker')

Label(text='Hello!')
with UntilDone():
    KeyPress()
    
exp.run()

# Instructions

- Participants need some guidance as to how they should perform the task
- This can be just text via `Label` states, but even better would be a visual description, and even better would be a full tutorial.
- ***NOTE: If you provide a tutorial, do not use items that you will use for the actual task!***

In [3]:
from smile.common import *

exp = Experiment(show_splash=False, resolution=(1024,768))

# Labels can be multi-line with markup!
with Parallel():
    Label(text="This is multi-line text aligned to the left", 
          font_size=50, text_size=(400, None),
          left=exp.screen.left)
    Label(text="This is multi-line text aligned to the right", 
          font_size=50, text_size=(400, None), 
          right=exp.screen.right, halign='right')
with UntilDone():
    KeyPress()
    
exp.run()

# Scaling to different screen sizes/densities

- Screen sizes and pixel densities can vary across devices (especially on laptops, tablets, and phones)
- But all sizes in Kivy/SMILE are defined by number of pixels
- We provide a means to scale across devices:

In [5]:
# load in smile states
from smile.common import *
from smile.scale import scale as s

# create an experiment instance (pick different resolutions)
resolution = (800,600)
#resolution = (1024,768)
exp = Experiment(show_splash=False, resolution=resolution,
                 scale_up=True, scale_down=True, scale_box=(400, 300))
Wait(.5)
with Parallel():
    # add a scaled rectangle
    Rectangle(width=s(250),height=s(250))
    
    # and a fixed size rectangle
    Rectangle(width=100, height=100, color='blue')
with UntilDone():
    KeyPress()

exp.run()

# Logging and Slogging

- SMILE states automatically log themselves, so you rarely will lose information if you forget to log it.
- It's still much easier to analyze a well-organized log file from your experiment.
- Make use of the `Log` state to save out data.
- All logs are stored in `.slog` files, which is short for SMILE Log.
- These are read in as a list of dictionaries (or a dict-list):

```python
from smile.log import log2dl
dl = log2dl('flanker_log.slog')
```

# Where have all the data gone?

- All the data files are stored in the `data` directory
  - NOTE: Even when you don't specify a subj, it saves to a `test000` directory, so you can delete that sometimes to save space.
- There is a hierarchical structure to the data directory:
  - Experiment -> SubjID -> Date/Time

In [7]:
!du -sh data

1.4M	data


In [14]:
!ls data/SMILE/test000/

20210930_121744  20210930_122106  20210930_152458  20211007_124549
20210930_121757  20210930_122230  20210930_152801  20211007_125947
20210930_121804  20210930_143329  20210930_152821  20211007_130009
20210930_121946  20210930_144041  20210930_153142  20211007_130034
20210930_121952  20210930_144537  20210930_155217  20211007_130046
20210930_122001  20210930_145759  20210930_155329  20211007_130056
20210930_122018  20210930_145925  20211007_123759  20211007_130122
20210930_122039  20210930_150609  20211007_124533


In [15]:
from smile.log import log2dl
dl = log2dl('data/SMILE/test000/20210930_121744/state_KeyPress_0.slog')
dl

[{'instantiation_filename': 'Turn on Debug Mode for this information',
  'instantiation_lineno': 0,
  'name': None,
  'start_time': 286271.466313349,
  'end_time': 286272.50535434205,
  'enter_time': 286271.282900669,
  'leave_time': 286272.506153025,
  'finalize_time': 286272.506269767,
  'base_time': 286271.466313349,
  'pressed': 'SPACEBAR',
  'press_time_time': 286272.50535434205,
  'press_time_error': 0.00024182800552807748,
  'correct': False,
  'rt': 1.039040993025992,
  'log_num': 0},
 {'instantiation_filename': 'Turn on Debug Mode for this information',
  'instantiation_lineno': 0,
  'name': None,
  'start_time': 286273.00535434205,
  'end_time': 286273.636040512,
  'enter_time': 286272.509124796,
  'leave_time': 286273.637198863,
  'finalize_time': 286273.637626568,
  'base_time': 286273.00535434205,
  'pressed': 'SPACEBAR',
  'press_time_time': 286273.636040512,
  'press_time_error': 0.00029039199580438435,
  'correct': False,
  'rt': 0.6306861699558794,
  'log_num': 0},
 {'

# Let's continue learning by building together!

- Remaining features to add:
  - Instructions
  - Loop over blocks and then trials
  - Verify logging

# List Gen Function

- NOTE: New addition for multiple blocks of trials

In [16]:
import random 
import copy

# define the conditions
conds = [{'condition': 'congruent',
          'direction': 'left',
          'stimulus': '<<<<<<<'
         },
         {'condition': 'congruent',
          'direction': 'right',
          'stimulus': '>>>>>>>'
         },
         {'condition': 'incongruent',
          'direction': 'left',
          'stimulus': '>>><>>>'
         },
         {'condition': 'incongruent',
          'direction': 'right',
          'stimulus': '<<<><<<'
         },
         {'condition': 'neutral',
          'direction': 'left',
          'stimulus': '===<==='
         },
         {'condition': 'neutral',
          'direction': 'right',
          'stimulus': '===>==='
         },]

# specify number of reps of these conditions
num_reps = 2
num_blocks = 3

# loop and create the blocks
blocks = []
for b in range(num_blocks):
    # loop and create the list
    trials = []
    for i in range(num_reps):
        # extend the trials with copies of the conditions
        trials.extend(copy.deepcopy(conds))
        
    # shuffle the trials
    random.shuffle(trials)
    
    # append the trials to the blocks
    blocks.append(trials)

print(blocks)

[[{'condition': 'congruent', 'direction': 'right', 'stimulus': '>>>>>>>'}, {'condition': 'congruent', 'direction': 'left', 'stimulus': '<<<<<<<'}, {'condition': 'neutral', 'direction': 'left', 'stimulus': '===<==='}, {'condition': 'neutral', 'direction': 'right', 'stimulus': '===>==='}, {'condition': 'incongruent', 'direction': 'left', 'stimulus': '>>><>>>'}, {'condition': 'incongruent', 'direction': 'right', 'stimulus': '<<<><<<'}, {'condition': 'neutral', 'direction': 'right', 'stimulus': '===>==='}, {'condition': 'incongruent', 'direction': 'left', 'stimulus': '>>><>>>'}, {'condition': 'congruent', 'direction': 'right', 'stimulus': '>>>>>>>'}, {'condition': 'congruent', 'direction': 'left', 'stimulus': '<<<<<<<'}, {'condition': 'incongruent', 'direction': 'right', 'stimulus': '<<<><<<'}, {'condition': 'neutral', 'direction': 'left', 'stimulus': '===<==='}], [{'condition': 'incongruent', 'direction': 'left', 'stimulus': '>>><>>>'}, {'condition': 'congruent', 'direction': 'right', 'st

# Goal for task

## High level structure

- Present instructions
- Loop over blocks
- Within each block, loop over trials

## Trial structure

- Present the correct stimulus as text on the screen
- Wait for a response
- Remove the stimulus
- Wait for an inter-stimulus interval
- Log the result of the trial

In [17]:
from smile.common import *
from smile.scale import scale as s

# config section
font_size = 75
resp_keys = ['F', 'J']
resp_map = {'left': 'F', 'right': 'J'}
ISI_dur = 0.5
ISI_jitter = 0.5
LOC_X_jitter = 200
LOC_Y_jitter = 100
inst_font_size = 25
inst_text = """[u][size=40]FLANKER INSTRUCTIONS[/size][/u]

In this task, you will see stimuli one at at time on the screen. 
    
Press ENTER key to continue."""

# create the experiment
exp = Experiment(name='FLANKER', show_splash=False, 
                 fullscreen=False,
                 resolution=(1024, 768), scale_box=(1024, 768))

@Subroutine
def Instruct(self):
    # show the instructions
    Label(text=inst_text, font_size=inst_font_size,
          text_size=(exp.screen.width*0.75, None),
          markup=True)
    with UntilDone():
        KeyPress(keys=['ENTER'])

@Subroutine
def Trial(self, block_num, trial_num, cur_trial):
    # pick the new stimulus location
    self.location = (jitter(self.exp.screen.center_x-LOC_X_jitter,
                            LOC_X_jitter*2),
                     jitter(self.exp.screen.center_y-LOC_Y_jitter,
                            LOC_Y_jitter*2))
    # present the stimulus
    stim = Label(text=cur_trial['stimulus'],
                 font_size=font_size,
                 center=self.location)
    with UntilDone():
        # make sure the stimulus has appeared on the screen
        Wait(until=stim.appear_time)
        
        # collect a response (with no timeout)
        kp = KeyPress(keys=resp_keys, 
                      base_time=stim.appear_time['time'],
                      correct_resp=Ref.object(resp_map)[cur_trial['direction']])
    
    # wait the ISI with jitter
    Wait(ISI_dur, jitter=ISI_jitter)

    # log the result of the trial
    Log(name='flanker', 
        log_dict=cur_trial,
        block_num=block_num,
        trial_num=trial_num,
        stim_on=stim.appear_time,
        resp=kp.pressed,
        resp_time=kp.press_time,
        rt=kp.rt,
        correct=kp.correct,
        location=self.location
       )

# Get the subj id information


# show the instructions
Instruct()
Wait(0.5)

# loop over the blocks
with Loop(blocks) as block:
    # make sure they are ready to continue
    Label(text='Press the ENTER key to\nstart the next block.', 
          font_size=font_size, halign='center')
    with UntilDone():
        KeyPress(keys=['ENTER'])

    # add in some delay before the start of the block
    Wait(ISI_dur, jitter=ISI_jitter)
    
    # loop over the trials
    with Loop(block.current) as trial:
        Trial(block.i, trial.i, trial.current)

# make sure they are ready to continue
Label(text='You are all done!!!\nPress the ENTER key to go celebrate.', 
      font_size=font_size, halign='center')
with UntilDone():
    KeyPress(keys=['ENTER'])

# run the experiment
exp.run()

# Read in some data from the logs

In [18]:
from smile.log import log2dl
dl = log2dl('data/FLANKER/test000/20211007_130711/log_flanker_0.slog')
dl

[{'block_num': 0,
  'trial_num': 0,
  'stim_on_time': 306569.269299902,
  'stim_on_error': 0.0,
  'resp': 'J',
  'resp_time_time': 306569.8923222975,
  'resp_time_error': 0.00022534752497449517,
  'rt': 0.6230223954771645,
  'correct': True,
  'location_0': 565.9039283500274,
  'location_1': 336.7582154358705,
  'log_time': 306570.79416547343,
  'condition': 'congruent',
  'direction': 'right',
  'stimulus': '>>>>>>>',
  'log_num': 0},
 {'block_num': 0,
  'trial_num': 1,
  'stim_on_time': 306570.799816691,
  'stim_on_error': 0.0,
  'resp': 'F',
  'resp_time_time': 306571.3687474585,
  'resp_time_error': 0.0003150884876959026,
  'rt': 0.5689307674765587,
  'correct': True,
  'location_0': 624.8102965078526,
  'location_1': 437.37237244048015,
  'log_time': 306572.2829832691,
  'condition': 'congruent',
  'direction': 'left',
  'stimulus': '<<<<<<<',
  'log_num': 0},
 {'block_num': 0,
  'trial_num': 2,
  'stim_on_time': 306572.298992042,
  'stim_on_error': 0.0,
  'resp': 'F',
  'resp_tim

In [20]:
import pandas as pd

df = pd.DataFrame(dl)
df.head()

,block_num,trial_num,stim_on_time,stim_on_error,resp,resp_time_time,resp_time_error,rt,correct,location_0,location_1,log_time,condition,direction,stimulus,log_num
0,0,0,306569.269300,0.0,J,306569.892322,0.000225,0.623022,True,565.903928,336.758215,306570.794165,congruent,right,>>>>>>>,0
1,0,1,306570.799817,0.0,F,306571.368747,0.000315,0.568931,True,624.810297,437.372372,306572.282983,congruent,left,<<<<<<<,0
2,0,2,306572.298992,0.0,F,306572.910505,0.000539,0.611512,True,469.474295,410.397244,306573.726506,neutral,left,===<===,0
3,0,3,306573.734621,0.0,J,306574.226776,0.000251,0.492155,True,394.057361,461.137053,306575.222020,neutral,right,===>===,0
4,0,4,306575.223116,0.0,J,306575.777729,0.000207,0.554613,False,694.416581,448.905599,306576.670761,incongruent,left,>>><>>>,0


## Assignment before next class

- Your SMILE experiment is due next Thursday!

### See you next week!!!